In [ ]:
# Load Data set

from google.colab import files
# use one hot encoding
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elliottjensen","key":"38a9322f7952b7d66dc295a731979a3f"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [ ]:
!kaggle competitions download -c ucsc-cse-164-spring-2023-final-project

 95% 96.0M/101M [00:03<00:00, 35.2MB/s]
100% 101M/101M [00:03<00:00, 27.4MB/s] 


In [ ]:
!unzip /content/ucsc-cse-164-spring-2023-final-project.zip

Streaming output truncated to the last 5000 lines.
  inflating: CSE164_2023/Unlabeled data/Unlabeled_55.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_550.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5500.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5501.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5502.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5503.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5504.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5505.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5506.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5507.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5508.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5509.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_551.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5510.jpeg  
  inflating: CSE164_2023/Unlabeled data/Unlabeled_5511.jpeg  
  inflating: CSE164_202

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from functools import partial
import copy

In [ ]:
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# x=tf.keras.utils.image_dataset_from_directory("/content/CSE164_2023/Test set",labels=None)
# print(x)

In [ ]:
# create data generators which augment data

from keras.applications.resnet import preprocess_input

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    # rescale = 1. / 255, # normalization
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    # vertical_flip = True,
    rotation_range = 30,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    validation_split = 0.2,
    # preprocessing_function=preprocess_input
    )


train_generator = data_generator.flow_from_directory(
    "/content/CSE164_2023/Train set",
    target_size =(64, 64),
    # batch_size = 100,
    classes = ["1","2", "3","4","5", "6", "7", "8", "9", "10"],
    shuffle = True,
    class_mode = 'categorical',
    seed = 42,
    subset='training')

validation_generator = data_generator.flow_from_directory(
    "/content/CSE164_2023/Train set",
    target_size =(64, 64),
    # batch_size = 100,
    classes = ["1","2", "3","4","5", "6", "7", "8", "9", "10"],
    shuffle = True,
    class_mode = 'categorical',
    seed = 42,
    subset='validation')

# Define a function that yields data and labels from the data_flow generator
def generator():
    for data, labels in train_generator:
        yield data, labels



Found 80 images belonging to 10 classes.
Found 20 images belonging to 10 classes.


In [ ]:
# view augmented  data

import matplotlib.pyplot as plt

class_names = [
    "boat",
    "truck",
    "car",
    "bird",
    "monkey",
    "horselike",
    "dog",
    "plane",
    "feline",
    "horse"
]

# Assuming you have a dataset generator named 'dataset_generator'

# Get a batch of data and labels from the generator
batch_x, batch_y = train_generator.next()

# Visualize each image in the batch along with its label
for i in range(len(batch_x)):
    image = batch_x[i]
    label = tf.argmax(batch_y[i])

    # Plot the image
    plt.imshow(image.astype("int"))
    plt.axis('off')

    # Set the title as the label
    plt.title('Label: {}'.format(class_names[label]))

    # Show the plot
    plt.show()

In [ ]:
# sequential model

# model = tf.keras.models.Sequential()

# ################## Your code begins here #####################
# # Add layers
# # call model.add multiplpe times until all necessary layers are added
# model.add(tf.keras.layers.Conv2D(64,3,activation="ReLU",input_shape=(64,64,3)))
# model.add(tf.keras.layers.Conv2D(128,3,activation="ReLU"))
# model.add(tf.keras.layers.Conv2D(128,3,activation="ReLU"))
# # model.add(tf.keras.layers.Conv2D(512,3,activation="ReLU"))

# model.add(tf.keras.layers.Flatten())

# model.add(tf.keras.layers.Dense(512,activation="relu"))
# model.add(tf.keras.layers.Dense(10,activation="softmax"))



In [ ]:
# Build Model

from tensorflow.keras import layers
# model = tf.keras.applications.EfficientNetB0

inputs=layers.Input(shape=(64,64,3))
# model = tf.keras.applications.EfficientNetB0(include_top=False,input_tensor=inputs,input_shape=(64,64,3),weights="imagenet")
# model = tf.keras.applications.resnet50(include_top=False,input_tensor=inputs,input_shape=(64,64,3),weights="imagenet")
inputs = tf.keras.Input(shape=(64, 64, 3))
# x = data_augmentation(inputs)
# x = tf.keras.applications.resnet50.preprocess_input(inputs)
# x = base_model(x, training=False)
# x = global_average_layer(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# outputs = prediction_layer(x)
# model = tf.keras.Model(inputs, outputs)
from tensorflow.keras.applications.resnet50 import ResNet50
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64 , 3))
# model.trainable = False
for layer in resnet.layers:
    layer.trainable = False
x = resnet.output
# x = tf.keras.applications.resNet50.preprocess_input(x)
x = layers.Flatten()(x)
# x=layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
# model.add(tf.keras.layers.Dropout(.2, input_shape=(2,))
# x=layers.BatchNormalization()(x)
x=layers.Dense(512,activation="relu")(x)
x=tf.keras.layers.Dropout(.5)(x)
# dropout
# different models; like transformers
# follow the exercise
# delete unlabled y true
outputs=layers.Dense(10,activation="softmax",name="pred")(x)
model = tf.keras.Model(inputs=resnet.input, outputs=outputs)
# model=tf.keras.Model(inputs,outputs,name="EfficientNet")

In [ ]:
# !pip install -U tensorflow-addons

In [ ]:
# Transformer
# import tensorflow as tf
import tensorflow_addons as tfa
# !pip install vit_keras
from vit_keras import vit

vit_model = vit.vit_b16(
        image_size = (64,64),
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 10)
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(64, activation = tfa.activations.gelu),
        tf.keras.layers.Dense(32, activation = tfa.activations.gelu),
        tf.keras.layers.Dense(10, 'softmax')
    ],
    name = 'vision_transformer')

In [ ]:
# Train Model

# opt = keras.optimizers.Adam(learning_rate = 0.01)
opt = tfa.optimizers.RectifiedAdam(learning_rate = 0.0001)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
# model.fit(x_train,y_train)
model.fit(train_generator, epochs=200,batch_size=64,validation_data=validation_generator, verbose=1)
# model.fit_generator(train_generator, epochs=200,validation_data=validation_generator, verbose=2)

Epoch 1/200
3/3 [==============================] - 32s 2s/step - loss: 2.5107 - accuracy: 0.0875 - val_loss: 2.3311 - val_accuracy: 0.0500
Epoch 2/200
3/3 [==============================] - 1s 216ms/step - loss: 2.4983 - accuracy: 0.1000 - val_loss: 2.3931 - val_accuracy: 0.0500
Epoch 3/200
3/3 [==============================] - 1s 265ms/step - loss: 2.5601 - accuracy: 0.0750 - val_loss: 2.3823 - val_accuracy: 0.0500
Epoch 4/200
3/3 [==============================] - 1s 217ms/step - loss: 2.4569 - accuracy: 0.1000 - val_loss: 2.3647 - val_accuracy: 0.0500
Epoch 5/200
3/3 [==============================] - 1s 217ms/step - loss: 2.5353 - accuracy: 0.0750 - val_loss: 2.3984 - val_accuracy: 0.0500
Epoch 6/200
3/3 [==============================] - 1s 245ms/step - loss: 2.5219 - accuracy: 0.0625 - val_loss: 2.2809 - val_accuracy: 0.0500
Epoch 7/200
3/3 [==============================] - 1s 231ms/step - loss: 2.4753 - accuracy: 0.1000 - val_loss: 2.3560 - val_accuracy: 0.0500
Epoch 8/200
3/3

In [ ]:
### END SUPERVISED LEARNING ###
### BEGIN UNSUPERVISED LEARNING ###



In [ ]:
# 4 cases unlabeled;labeled; entroopy ture/ false
# copying the exercise
# tuning parameters
# chat gpt pseudo labeling

# dynamically update training data
#model.fit(updated)
# modeol . call enxt cohort
# concatenate; not replace

class SemisupervisedModel(keras.Model):
    def __init__(self, model, entropy=False, lamda=0.1): # tune lambda
        super().__init__()

        self.model = model
        self.cce = tf.keras.losses.CategoricalCrossentropy()

        self.entropy = entropy
        self.lamda = lamda

    def compile(self, optimizer, **kwargs):
        super().compile(**kwargs)

        self.optimizer = optimizer
        self.accuracy_tracker = keras.metrics.Accuracy()
        self.loss_tracker = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker, self.accuracy_tracker]
        # return [self.loss_tracker]

    def call(self, images, training=True):
        return self.model(images, training=training)

    def semisupervised_loss(self, labeled_y_true, labeled_y_pred, unlabeled_y_true, unlabeled_y_pred):
        loss = self.cce(labeled_y_true, labeled_y_pred)
        if self.entropy:
            loss += -self.lamda * tf.reduce_mean(tf.reduce_sum(unlabeled_y_pred * tf.math.log(unlabeled_y_pred + 1e-12), axis=1))
        return loss

    def train_step(self, data):
        (unlabeled_images, unlabeled_y_true), (labeled_images, labeled_y_true) = data

        with tf.GradientTape() as tape:
            labeled_y_pred = self.call(labeled_images, training=True)
            unlabeled_y_pred = self.call(unlabeled_images, training=True)
            loss = self.semisupervised_loss(labeled_y_true, labeled_y_pred, unlabeled_y_true, unlabeled_y_pred)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.loss_tracker.update_state(loss)

        self.accuracy_tracker.update_state(tf.math.argmax(unlabeled_y_true, axis=1), tf.math.argmax(unlabeled_y_pred, axis=1))
        self.accuracy_tracker.update_state(tf.math.argmax(labeled_y_true, axis=1), tf.math.argmax(labeled_y_pred, axis=1))

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        images, labels = data
        y_pred = self.call(images, training=False)

        loss = self.cce(labels, y_pred)
        self.loss_tracker.update_state(loss)

        self.accuracy_tracker.update_state(tf.math.argmax(labels, axis=1), tf.math.argmax(y_pred, axis=1))

        return {m.name: m.result() for m in self.metrics}

In [ ]:
# dataset = tf.data.Dataset.from_generator( train_generator)

In [ ]:
semimodel = SemisupervisedModel(
    model,entropy=True
    # keras.Sequential(
    # [
    #     keras.Input(shape=(64,64,3)),
    #     layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    #     layers.MaxPooling2D(pool_size=(2, 2)),
    #     layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    #     layers.MaxPooling2D(pool_size=(2, 2)),
    #     layers.Flatten(),
    #     layers.Dropout(0.5),
    #     layers.Dense(10, activation="softmax"),
    # ]
    # ),
    # entropy=False
)

In [ ]:
# defines labeled data

# labeled_x_train, labeled_y_train = train_generator.next()
x_test_names = []
unlabeled_x_train = []

from PIL import Image
for i in range(1,10001):
    x_test_names.append(str(i) + ".jpeg")
    img = Image.open(r"/content/CSE164_2023/Unlabeled data/Unlabeled_" +str(i)+".jpeg")
    img = np.array(img)
    # print("img.shape = ", img.shape)
    # if(img.shape[0] != 64 or img.shape[1] != 64 or img.shape [2] != 3):
    #     print("wtf")

    unlabeled_x_train.append(img)
unlabeled_x_train = np.array(unlabeled_x_train)
print(unlabeled_x_train.shape)

unlabeled_y_train = np.ones((10000,1))


(10000, 64, 64, 3)


In [ ]:
# shuffles labeled and unlabled

# unlabeled_x_train = tf.keras.utils.image_dataset_from_directory("/content/CSE164_2023/Unlabeled data",labels=None)
labeled_x_train,labeled_y_train = train_generator[0][0], train_generator[0][1]
x_val,y_val = validation_generator[0][0], validation_generator[0][1]
# unlabeled_y_train = np.ones(())

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 500
# total_train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# total_train_dataset = total_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

labeled_train_dataset = tf.data.Dataset.from_tensor_slices((labeled_x_train, labeled_y_train))
labeled_train_dataset = labeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val,y_val))
val_dataset = val_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

# validation_set = tf.data.Dataset.from_tensor_slices((x_val,y_val))
# validation_set = tf.data.Dataset.zip(validation_set)

unlabeled_train_dataset = tf.data.Dataset.from_tensor_slices((unlabeled_x_train, unlabeled_y_train))
unlabeled_train_dataset = unlabeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset = tf.data.Dataset.zip((unlabeled_train_dataset, labeled_train_dataset)) #.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

#

# test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
# test_dataset = test_dataset.batch(BATCH_SIZE)

# BATCH_SIZE = 64
# SHUFFLE_BUFFER_SIZE = 500
# total_train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# total_train_dataset = total_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# # unlabeled_y_train = np.ones(labeled_y.shape)
# labeled_train_dataset = tf.data.Dataset.from_tensor_slices((labeled_x_train, labeled_y_train))
# print(labeled_y_train)
# labeled_train_dataset = labeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# unlabeled_train_dataset = tf.data.Dataset.from_tensor_slices((unlabeled_x_train,unlabeled_y_train))
# unlabeled_train_dataset = unlabeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

# print(unlabeled_train_dataset)
# print(labeled_train_dataset)

# train_dataset = tf.data.Dataset.zip((unlabeled_train_dataset, labeled_train_dataset))

# right now modifying the superivsed model so that it accepts generators

In [ ]:
# Create a dataset from the generator
# dataset = tf.data.Dataset.from_generator(generator, output_signature=(
#     tf.TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32),
#     tf.TensorSpec(shape=(None, 10), dtype=tf.float32)
# ))
# train_dataset = dataset

In [ ]:
semi_model = SemisupervisedModel(
    keras.Sequential(
    [
        keras.Input(shape=(64,64,3)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
    ),
    entropy=True
)

In [ ]:
# Train Semi model

epochs = 100
# opt = tfa.optimizers.RectifiedAdam(learning_rate = 0.0001)
opt =keras.optimizers.Adam(learning_rate=0.01)
semimodel.compile(optimizer=opt, metrics=["accuracy"])
# print(train_dataset.shape)

# train_dataset = tf.data.Dataset.zip((train_dataset, train_dataset))
semimodel.fit(train_dataset ,epochs=epochs, batch_size=32,validation_data = val_dataset)

Epoch 1/100
1/1 [==============================] - 69s 69s/step - loss: 0.2853 - accuracy: 0.3542 - val_loss: 53.4191 - val_accuracy: 0.2000
Epoch 2/100
1/1 [==============================] - 0s 454ms/step - loss: 0.8965 - accuracy: 0.3542 - val_loss: 59.0705 - val_accuracy: 0.2000
Epoch 3/100
1/1 [==============================] - 0s 375ms/step - loss: 0.2117 - accuracy: 0.3750 - val_loss: 63.2195 - val_accuracy: 0.1500
Epoch 4/100
1/1 [==============================] - 0s 473ms/step - loss: 0.0142 - accuracy: 0.4167 - val_loss: 65.3002 - val_accuracy: 0.1000
Epoch 5/100
1/1 [==============================] - 0s 417ms/step - loss: 0.0216 - accuracy: 0.4271 - val_loss: 69.3662 - val_accuracy: 0.1000
Epoch 6/100
1/1 [==============================] - 0s 430ms/step - loss: 0.2292 - accuracy: 0.3854 - val_loss: 69.9763 - val_accuracy: 0.1000
Epoch 7/100
1/1 [==============================] - 0s 433ms/step - loss: 0.0150 - accuracy: 0.4479 - val_loss: 67.5830 - val_accuracy: 0.1000
Epoch 8

In [ ]:
# labeled_x_train, labeled_y_train = train_generator.next()
# x_test_names = []
# unlabeled_x_train = []
# for i in range(1,10001):
#     x_test_names.append(str(i) + ".jpeg")
#     img = Image.open(r"/content/CSE164_2023/Unlabeled data/Unlabeled_" +str(i)+".jpeg")
#     img = np.array(img)
#     # print("img.shape = ", img.shape)
#     # if(img.shape[0] != 64 or img.shape[1] != 64 or img.shape [2] != 3):
#     #     print("wtf")
#     # if(img.shape!=(64,64,3)):
#         # print('wtf')
#     unlabeled_x_train.append(img)
# unlabeled_x_train = np.array(unlabeled_x_train)
# print(unlabeled_x_train.shape)



In [ ]:
unlabeled_x_train = tf.keras.utils.image_dataset_from_directory("/content/CSE164_2023/Unlabeled data",labels=None)


BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 500
# total_train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# total_train_dataset = total_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# unlabeled_y_train = np.ones(labeled_y.shape)
labeled_train_dataset = tf.data.Dataset.from_tensor_slices((labeled_x_train, labeled_y_train))
print(labeled_y_train)
labeled_train_dataset = labeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
unlabeled_train_dataset = tf.data.Dataset.from_tensor_slices((unlabeled_x_train,unlabeled_y_train))
unlabeled_train_dataset = unlabeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

print(unlabeled_train_dataset)
print(labeled_train_dataset)

train_dataset = tf.data.Dataset.zip((unlabeled_train_dataset, labeled_train_dataset))

In [ ]:
train_dataset

In [ ]:
epochs = 100

semimodel.compile(optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
# print(train_dataset.shape)
semimodel.fit(train_dataset, epochs=epochs)

In [ ]:
# Get the number of samples in the validation generator
num_samples = len(validation_generator)

# Reset the generator to the beginning
validation_generator.reset()

# Iterate over the generator and print the labels
for i in range(num_samples):
    # Retrieve a batch of data and labels
    batch_x, batch_y = validation_generator.next()

    # Print the labels
    print(batch_y)

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory("/content/CSE164_2023/Test set",image_size=(64,64),labels=None)
len(list(test_dataset))


# x_test=test_dataset
# print(len(x_test))
# count = 0
# for image in test_dataset:
#     count +=1
# print(count)

In [ ]:
# Make Submission Kaggle File

import pandas as pd
from PIL import Image
x_test_names = []
x_test = []
for i in range(1,10001):
    x_test_names.append(str(i) + ".jpeg")
    img = Image.open(r"/content/CSE164_2023/Test set/" +str(i)+".jpeg")
    img = np.array(img)
    # print("img.shape = ", img.shape)
    # if(img.shape[0] != 64 or img.shape[1] != 64 or img.shape [2] != 3):
    #     print("wtf")
    # if(img.shape!=(64,64,3)):
        # print('wtf')
    x_test.append(img)
x_test = np.array(x_test)
# print(x_test_names[-1])



In [ ]:
# print("x_test_names[0] = ", x_test_names[0])
# p = np.argmax(model.predict(x_test[3]))
test_preds = np.argmax(model.predict(x_test),axis=1)
print(test_preds.shape)
copy = pd.read_csv("/content/CSE164_2023/submission_samples.csv")
copy["label"] = test_preds
pd.DataFrame(copy).to_csv('submission.csv', index=False)


313/313 [==============================] - 19s 53ms/step
(10000,)


In [ ]:
def create_submission_file(model):
  name2idx = {}
  sample_submission = pd.read_csv('/content/CSE164_2023/submission_samples.csv')
  filename_order = sample_submission['Image_id'].values
  for i in range(len(filename_order)):
    name2idx[filename_order[i]] = i
#   x_test = tf.image.resize(x_test,[224,224])

  # Google colab reads the files in a different order than the answer file was created.
  # This is done to preserve the file order.
  print(len(x_test))
  result_dict = {'Image_id': [None]*len(x_test),
                'label':[None]*len(x_test)}
  test_preds = np.argmax(model.predict(x_test),axis=-1)

  for i in range(len(test_preds)):
    # test_image_name = test_preds[i]
    test_image_name = x_test_names[i]
    result_dict['Image_id'][name2idx[test_image_name]] = test_image_name
    result_dict['label'][name2idx[test_image_name]] = test_preds[i]

  pd.DataFrame(result_dict).to_csv('submission.csv', index=False)
create_submission_file(model)

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory("/content/CSE164_2023/Test set",image_size=(64,64),labels=None)
# list(test_dataset)[0]
# model.predict(test_dataset)
# import pandas as pd

# pd.dataframe(test_dataset)
# input_arr = tf.keras.utils.img_to_array(test_dataset)
# input_arr = np.array([input_arr])  # Convert single image to a batch.
# print(test_dataset)
# test_dataset=input_arr

# test_preds = model.predict(x)
# test_presd = model.predict(test_dataset)
# image = tf.keras.utils.load_img(image_path)
# input_arr = tf.keras.utils.img_to_array(image)
# input_arr = np.array([input_arr])  # Convert single image to a batch.
# predictions = model.predict(input_arr
# test_preds = np.argmax(model.predict(test_dataset),axis=1)

Found 10000 files belonging to 1 classes.


In [ ]:
# score = model.evaluate(x_test,y_test, verbose=0)
# print("Test loss:", score[0])
# print("Test accuracy:", score[1])

In [ ]:
# create submission
import pandas as pd

# test_preds = semi_model.predict(test_dataset)
# test_dataset = tf.keras.utils.image_dataset_from_directory("/content/CSE164_2023/Test set",labels=None)
test_preds = np.argmax(model.predict(test_dataset),axis=1)
copy = pd.read_csv("/content/CSE164_2023/submission_samples.csv")
copy["label"] = test_preds
pd.DataFrame(copy).to_csv('submission.csv', index=False)


### END ###

313/313 [==============================] - 21s 57ms/step


In [ ]:
# make submission
import pandas as pd
# test_preds = semi_model.predict(test_dataset)
# print(test_preds.shape)
# print(test_preds)
# print(test_preds[0])

test_preds = np.argmax(model.predict(test_dataset),axis=1)
copy = pd.read_csv("/content/CSE164_2023/submission_samples.csv")
copy["label"] = test_preds
pd.DataFrame(copy).to_csv('submission.csv', index=False)

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (64, 64, 3)

# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

train_dataset =  tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [ ]:
selected_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
num_classes = 10
y_train_index = np.argmax(y_train, axis=1)
y_test_index = np.argmax(y_test, axis=1)

x_train = x_train[np.isin(y_train_index, selected_classes)]
y_train = y_train[np.isin(y_train_index, selected_classes)]
x_test = x_test[np.isin(y_test_index, selected_classes)]
y_test = y_test[np.isin(y_test_index, selected_classes)]
y_train = y_train[:, selected_classes]
y_test = y_test[:, selected_classes]
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)
# print(x_train.shape[0], "train samples")
# print(x_test.shape[0], "test samples")
# print(y_train.shape[0], "train labels")
# print(y_test.shape[0], "test labels")
print(y_train)

In [ ]:
# the way they split is different; our project already ahs it split
unlabeld_ratio = 0.999
unlabeled_indices = np.random.uniform(size=y_train.shape[0]) <= unlabeld_ratio
labeled_indices = np.logical_not(unlabeled_indices)
labeled_x_train = x_train[labeled_indices]
labeled_y_train = y_train[labeled_indices]
unlabeled_x_train = x_train[unlabeled_indices]
unlabeled_y_train = y_train[unlabeled_indices]

In [ ]:
# unlabeled_x_train = "/content/CSE164_2023/Unlabeled data"

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 500
total_train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
total_train_dataset = total_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

labeled_x_train = x_train
labeled_y_train = y_train
labeled_train_dataset = tf.data.Dataset.from_tensor_slices((labeled_x_train, labeled_y_train))
labeled_train_dataset = labeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# unlabeled_train_dataset = tf.data.Dataset.from_tensor_slices((unlabeled_x_train, unlabeled_y_train))
# unlabeled_train_dataset = unlabeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# train_dataset = tf.data.Dataset.zip((unlabeled_train_dataset, labeled_train_dataset)) #.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
train_dataset = labeled_train_dataset
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
class SemisupervisedModel(keras.Model):
    def __init__(self, model, entropy=False, lamda=0.1):
        super().__init__()

        self.model = model
        self.cce = tf.keras.losses.CategoricalCrossentropy()

        self.entropy = entropy
        self.lamda = lamda

    def compile(self, optimizer, **kwargs):
        super().compile(**kwargs)

        self.optimizer = optimizer
        self.accuracy_tracker = keras.metrics.Accuracy()
        self.loss_tracker = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker, self.accuracy_tracker]
        # return [self.loss_tracker]

    def call(self, images, training=True):
        return self.model(images, training=training)

    def semisupervised_loss(self, labeled_y_true, labeled_y_pred, unlabeled_y_true, unlabeled_y_pred):
        loss = self.cce(labeled_y_true, labeled_y_pred)
        if self.entropy:
            loss += -self.lamda * tf.reduce_mean(tf.reduce_sum(unlabeled_y_pred * tf.math.log(unlabeled_y_pred + 1e-12), axis=1))
        return loss

    def train_step(self, data):
        (unlabeled_images, unlabeled_y_true), (labeled_images, labeled_y_true) = data

        with tf.GradientTape() as tape:
            labeled_y_pred = self.call(labeled_images, training=True)
            unlabeled_y_pred = self.call(unlabeled_images, training=True)
            loss = self.semisupervised_loss(labeled_y_true, labeled_y_pred, unlabeled_y_true, unlabeled_y_pred)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.loss_tracker.update_state(loss)

        self.accuracy_tracker.update_state(tf.math.argmax(unlabeled_y_true, axis=1), tf.math.argmax(unlabeled_y_pred, axis=1))
        self.accuracy_tracker.update_state(tf.math.argmax(labeled_y_true, axis=1), tf.math.argmax(labeled_y_pred, axis=1))

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        images, labels = data
        y_pred = self.call(images, training=False)

        loss = self.cce(labels, y_pred)
        self.loss_tracker.update_state(loss)

        self.accuracy_tracker.update_state(tf.math.argmax(labels, axis=1), tf.math.argmax(y_pred, axis=1))

        return {m.name: m.result() for m in self.metrics}

In [ ]:
model = SemisupervisedModel(
    keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
    ),
    entropy=False
)


In [ ]:
epochs = 100

model.compile(optimizer=keras.optimizers.Adam(), metrics=["accuracy"])

model.fit(train_dataset, epochs=epochs)

In [ ]:
score = model.evaluate(test_dataset, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

In [ ]:
semi_model = SemisupervisedModel(
    keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
    ),
    entropy=True
)

In [ ]:
epochs = 100

model.compile(optimizer=keras.optimizers.Adam(), metrics=["accuracy"])


model.fit(train_dataset, epochs=epochs)

In [ ]:
score = semi_model.evaluate(test_dataset, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


In [ ]:
# make submission
import pandas as pd
# test_preds = semi_model.predict(test_dataset)
# print(test_preds.shape)
# print(test_preds)
# print(test_preds[0])
test_preds = np.argmax(model.predict(test_dataset),axis=1)
copy = pd.read_csv("/content/CSE164_2023/submission_samples.csv")
copy["label"] = test_preds
pd.DataFrame(copy).to_csv('submission.csv', index=False)

In [ ]:
# create submission
# test_preds = semi_model.predict(test_dataset)
test_dataset = tf.keras.utils.image_dataset_from_directory("/content/CSE164_2023/Test set")
test_preds = np.argmax(model.predict(test_dataset),axis=1)
copy = pd.read_csv("/content/CSE164_2023/submission_samples.csv")
copy["label"] = test_preds
pd.DataFrame(copy).to_csv('submission.csv', index=False)

In [ ]:
# semi super vised copy
class SemisupervisedModel(keras.Model):
    def __init__(self, model, entropy=False, lamda=0.1):
        super().__init__()

        self.model = model
        self.cce = tf.keras.losses.CategoricalCrossentropy()

        self.entropy = entropy
        self.lamda = lamda

    def compile(self, optimizer, **kwargs):
        super().compile(**kwargs)

        self.optimizer = optimizer
        self.accuracy_tracker = keras.metrics.Accuracy()
        self.loss_tracker = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker, self.accuracy_tracker]
        # return [self.loss_tracker]

    def call(self, images, training=True):
        return self.model(images, training=training)

    def semisupervised_loss(self, labeled_y_true, labeled_y_pred, unlabeled_y_true, unlabeled_y_pred):
        loss = self.cce(labeled_y_true, labeled_y_pred)
        if self.entropy:
            loss += -self.lamda * tf.reduce_mean(tf.reduce_sum(unlabeled_y_pred * tf.math.log(unlabeled_y_pred + 1e-12), axis=1))
        return loss

    def train_step(self, data):
        (unlabeled_images, unlabeled_y_true), (labeled_images, labeled_y_true) = data

        with tf.GradientTape() as tape:
            labeled_y_pred = self.call(labeled_images, training=True)
            unlabeled_y_pred = self.call(unlabeled_images, training=True)
            loss = self.semisupervised_loss(labeled_y_true, labeled_y_pred, unlabeled_y_true, unlabeled_y_pred)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.loss_tracker.update_state(loss)

        self.accuracy_tracker.update_state(tf.math.argmax(unlabeled_y_true, axis=1), tf.math.argmax(unlabeled_y_pred, axis=1))
        self.accuracy_tracker.update_state(tf.math.argmax(labeled_y_true, axis=1), tf.math.argmax(labeled_y_pred, axis=1))

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        images, labels = data
        y_pred = self.call(images, training=False)

        loss = self.cce(labels, y_pred)
        self.loss_tracker.update_state(loss)

        self.accuracy_tracker.update_state(tf.math.argmax(labels, axis=1), tf.math.argmax(y_pred, axis=1))

        return {m.name: m.result() for m in self.metrics}